# 🤝 سیستم‌های گردش کار چندعاملی سازمانی (.NET)

## 📋 اهداف یادگیری

این دفترچه نحوه ساخت سیستم‌های چندعاملی پیشرفته در سطح سازمانی را با استفاده از Microsoft Agent Framework در .NET و مدل‌های GitHub نشان می‌دهد. شما یاد خواهید گرفت که چندین عامل تخصصی را از طریق گردش کارهای ساختاریافته هماهنگ کنید و از ویژگی‌های سازمانی .NET برای راه‌حل‌های آماده تولید بهره ببرید.

**قابلیت‌های چندعاملی سازمانی که خواهید ساخت:**
- 👥 **همکاری عامل‌ها**: هماهنگی نوع-ایمن عامل‌ها با اعتبارسنجی در زمان کامپایل
- 🔄 **هماهنگی گردش کار**: تعریف گردش کار به صورت اعلامی با الگوهای async در .NET
- 🎭 **تخصص نقش‌ها**: شخصیت‌ها و حوزه‌های تخصصی عامل‌ها با نوع قوی
- 🏢 **یکپارچه‌سازی سازمانی**: الگوهای آماده تولید با نظارت و مدیریت خطا

## ⚙️ پیش‌نیازها و تنظیمات

**محیط توسعه:**
- .NET 9.0 SDK یا بالاتر
- Visual Studio 2022 یا VS Code با افزونه C#
- اشتراک Azure (برای عامل‌های پایدار)

**بسته‌های مورد نیاز NuGet:**
```xml
<PackageReference Include="Microsoft.Extensions.AI.Abstractions" Version="9.9.0" />
<PackageReference Include="Azure.AI.Agents.Persistent" Version="1.2.0-beta.4" />
<PackageReference Include="Azure.Identity" Version="1.15.0" />
<PackageReference Include="System.Linq.Async" Version="6.0.3" />
<PackageReference Include="Microsoft.Extensions.AI" Version="9.8.0" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
```


In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI Abstractions
// Core AI abstraction layer providing standardized interfaces for multi-agent systems
// Essential for building type-safe, enterprise-grade agent orchestration
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.9.1"

Installed Packages Microsoft.Extensions.AI.Abstractions, 9.9.1

In [2]:
// 📦 Enterprise Multi-Agent Dependencies
// Azure.AI.Agents.Persistent: Enterprise agent management with state persistence and Azure integration
// Azure.Identity: Secure authentication for Azure services in multi-agent workflows
// System.Linq.Async: High-performance asynchronous LINQ operations for agent data processing
// Microsoft.Extensions.AI: Core AI service abstractions for agent coordination
// DotNetEnv: Secure environment variable management for multi-agent configuration
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.4"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: Microsoft.Extensions.AI, 9.9.1"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: OpenTelemetry.Api, 1.0.0"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.4 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.9.1 OpenTelemetry.Api, 1.0.1 System.Linq.Async, 6.0.3

In [3]:

#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3

In [4]:
// 🔗 Local Assembly Reference: Microsoft Agents AI OpenAI
// OpenAI-compatible integration for GitHub Models in multi-agent systems
// Enables high-performance language model access for agent communication and reasoning
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [5]:
// 🔄 Local Assembly Reference: Microsoft Agents Workflows
// Advanced workflow orchestration engine for complex multi-agent coordination
// Provides declarative workflow definition and type-safe agent interaction patterns
// #r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.2"

Error: Microsoft.Agents.AI version 1.0.0-preview.251001.2 cannot be added because version 1.0.0-preview.251001.3 was added previously.

In [6]:
// 🔌 NuGet Package Reference: OpenAI Integration (Preview)
// Preview version providing GitHub Models compatibility for multi-agent workflows
// Enables seamless integration with GitHub's model inference service for agent communication
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using OpenAI;

In [9]:
 using DotNetEnv;

In [10]:
Env.Load("../../../.env");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string REVIEWER_NAME = "Concierge";
const string REVIEWER_INSTRUCTIONS = @"""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """;

In [15]:
const string FRONTDESK_NAME = "FrontDesk";
const string FRONTDESK_INSTRUCTIONS = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [16]:

ChatClientAgentOptions frontdeskAgentOptions = new(name: FRONTDESK_NAME, instructions: FRONTDESK_INSTRUCTIONS);
ChatClientAgentOptions reviewerAgentOptions = new(name: REVIEWER_NAME, instructions: REVIEWER_INSTRUCTIONS);

In [17]:
AIAgent reviewerAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    reviewerAgentOptions);
AIAgent frontdeskAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    frontdeskAgentOptions);

In [18]:
var workflow = new WorkflowBuilder(frontdeskAgent)
            .AddEdge(frontdeskAgent, reviewerAgent)
            .Build();

In [19]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, new ChatMessage(ChatRole.User, "I would like to go to Paris."));

In [20]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));

In [21]:
string strResult = "";

In [22]:
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
            if (evt is AgentRunUpdateEvent executorComplete)
            {
                strResult += executorComplete.Data;
                Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
            }
}

83b7c6d5c52d4dd39d39a753a9fe1d4a: 
83b7c6d5c52d4dd39d39a753a9fe1d4a: Visit
83b7c6d5c52d4dd39d39a753a9fe1d4a:  the
83b7c6d5c52d4dd39d39a753a9fe1d4a:  Eiffel
83b7c6d5c52d4dd39d39a753a9fe1d4a:  Tower
83b7c6d5c52d4dd39d39a753a9fe1d4a:  for
83b7c6d5c52d4dd39d39a753a9fe1d4a:  iconic
83b7c6d5c52d4dd39d39a753a9fe1d4a:  views
83b7c6d5c52d4dd39d39a753a9fe1d4a:  of
83b7c6d5c52d4dd39d39a753a9fe1d4a:  the
83b7c6d5c52d4dd39d39a753a9fe1d4a:  city
83b7c6d5c52d4dd39d39a753a9fe1d4a: .
83b7c6d5c52d4dd39d39a753a9fe1d4a: 
83b7c6d5c52d4dd39d39a753a9fe1d4a: 
63600cef458c48b0a1e525bde5511110: 
63600cef458c48b0a1e525bde5511110: This
63600cef458c48b0a1e525bde5511110:  recommendation
63600cef458c48b0a1e525bde5511110:  is
63600cef458c48b0a1e525bde5511110:  not
63600cef458c48b0a1e525bde5511110:  approved
63600cef458c48b0a1e525bde5511110: .
63600cef458c48b0a1e525bde5511110:  While
63600cef458c48b0a1e525bde5511110:  the
63600cef458c48b0a1e525bde5511110:  Eiffel
63600cef458c48b0a1e525bde5511110:  Tower
63600cef458c48

In [23]:
strResult

Visit the Eiffel Tower for iconic views of the city.This recommendation is not approved. While the Eiffel Tower is a must-see for many travelers, it is often crowded and heavily tourist-focused. To refine the recommendation, consider suggesting an experience that immerses travelers in the local culture, perhaps by exploring lesser-known neighborhoods, local markets, or hidden gems that showcase the authentic lifestyle of the city.


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
